In [1]:
from dicomUtils import *
from generateIndexFile import *
import matplotlib.pyplot as plt

def imshow_15(sup_title, images, titles):
    fig=plt.figure(figsize=(50, 25))
    columns = 5
    rows = 3
    for i in range(len(images)):
        img = images[i]
        ax=fig.add_subplot(rows, columns, i+1)
#         ax.title.set_text(titles[i])#('{:.4f}'.format(titles[i]))

        ax.set_title(titles[i], fontsize = 20) 

        plt.axis('off')
        plt.imshow(img,cmap='gray')
#     fig.suptitle(sup_title, fontsize=16)
    fig.tight_layout(rect=[0, 0.03, 1, 0.95])
#     plt.show()
    plt.savefig(sup_title)
def imshow_25(sup_title, images, titles):
    fig=plt.figure(figsize=(16, 16))
    columns = 5
    rows = 5
    for i in range(len(images)):
        img = images[i]
        ax=fig.add_subplot(rows, columns, i+1)
        ax.title.set_text(titles[i])#('{:.4f}'.format(titles[i]))
        plt.axis('off')
        plt.imshow(img,cmap='gray')
    fig.suptitle(sup_title, fontsize=16)
    fig.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()
    plt.savefig(sup_title)

In [10]:
from transManager_pb2 import *
from dicomUtils import *
from ast import literal_eval
def build_volume_struct_from_files(rf_path):
    vsc = []
    tn_pre = 'report/' + rf_path.split('/')[-2]
    with open(rf_path, 'r') as rf:
        record_lines = rf.readlines()[3:]
        record_num = int(len(record_lines) / 2)
        for i in range(record_num):
            infos = record_lines[2*i].split('/')
            scores_np = np.array(record_lines[2*i+1].split('/')).astype(np.float)
            simg = cv2.imread(path.join(tn_pre, infos[0], 'full.png'), 0)
            spacing = literal_eval(infos[4])
            vsc.append(volumeResponse.volumeInfo(folder_name=infos[0], \
                dims = literal_eval(infos[1]),\
                orientation = literal_eval(infos[2]),\
                resolution = [spacing[0],spacing[1], float(infos[5])],\
                volume_loc_range = float(infos[6]),\
                sample_img = simg.tobytes(),\
                scores = volumeResponse.scoreInfo(rgroup_id=int(infos[3]),\
                        raw_score=scores_np[:-3],\
                        raw_rank_score = [],\
                        num_score=scores_np[-3],\
                        tags_score=scores_np[-2],\
                        mask_score=scores_np[-1])))
    return vsc

In [3]:
vsc = build_volume_struct_from_files('../data/PACS/IRB02/index.txt')

In [ ]:
from collections import namedtuple
volumeStruct = namedtuple("volumeStruct", "name sample_img cut_group scores_raw scores_norm")  

In [ ]:
from ast import literal_eval
def build_volume_struct_from_files(rf_path):
    vsc = []
    tn_pre = 'report/' + rf_path.split('/')[-2]
    with open(rf_path, 'r') as rf:
        record_lines = rf.readlines()[3:]
        record_num = int(len(record_lines) / 2)
        for i in range(record_num):
            infos = record_lines[2*i].split('/')
            scores = record_lines[2*i+1].split('/')
            dims = literal_eval(infos[1])
            simg = cv2.imread(path.join(tn_pre, infos[0], 'full.png'), 0)
            vsc.append(volumeStruct(name=infos[0], sample_img=simg, cut_group=int(infos[3]), scores_raw = np.array(scores).astype(np.float), scores_norm=None))
    return vsc

In [ ]:
vsc = build_volume_struct_from_files('../data/PACS/IRB02/index.txt')

In [4]:
vol_cut_group = []
for vs in vsc:
    vol_cut_group.append(vs.scores.rgroup_id)
unique_group = np.unique(vol_cut_group)
grouped_vsc = {}
for gid in unique_group:
    grouped_vsc[gid] = [vs for vs in vsc if vs.scores.rgroup_id == gid]

In [5]:
param_num = len(vsc[0].scores.raw_score)
param_num

18

In [6]:
grouped_vsc_sorted = {}
for gvsc_id in grouped_vsc:
    grouped_vsc_sorted[gvsc_id] = [vs for vs in sorted(grouped_vsc.get(gvsc_id), key=lambda vs: np.mean(vs.scores.raw_score), reverse=True)]

In [11]:
default_sel_percent = 0.8
grouped_vsc_sorted_norm = {}
for gvsc_id in grouped_vsc:
    gsvsc = grouped_vsc_sorted[gvsc_id]
    
    max_vs_local = []
    for i in range(param_num):
        max_vs_local.append(np.max([vs.scores.raw_score[i] for vs in gsvsc]))
    
    for i in range(len(gsvsc)):
        gsvsc[i].scores.raw_rank_score.extend([u/(v+0.0001) for u,v in zip(gsvsc[i].scores.raw_score,max_vs_local)])
        
    sel_num = int(len(gsvsc) * default_sel_percent)
    sel_gsvc = gsvsc[:sel_num]
    rule_out_gsvc = gsvsc[sel_num:]
    
    
    #sort_mean_sel_gsvc = [vs for vs in sorted(sel_gsvc, key=lambda vs: np.mean([u/(v+0.001) for u,v in zip(vs.scores_raw,max_vs_local)]), reverse=True)]
    
    sort_mean_sel_gsvc = [vs for vs in sorted(sel_gsvc, key=lambda vs: np.mean(vs.scores.raw_rank_score), reverse=True)]
    grouped_vsc_sorted_norm[gvsc_id] = sort_mean_sel_gsvc + rule_out_gsvc

In [22]:
for gid in unique_group:
    titles = [ str(round(np.mean(vs.scores.raw_score), 2))+'/'+str(round(np.mean(vs.scores.raw_rank_score), 2)) + '\n' + vs.folder_name for vs in grouped_vsc_sorted_norm[gid]]
    print(len(titles))
    #     imshow_15('group ' +str(gid),[np.frombuffer(vs.sample_img, np.dtype(np.int8))  for vs in grouped_vsc_sorted_norm[gid]], titles)

10
15


In [24]:
res_lst = []
for gid in unique_group:
    for vs in grouped_vsc_sorted_norm[gid]:
        vs.scores.rank_score = np.mean(vs.scores.raw_rank_score)
        res_lst.append(vs)
len(res_lst)

25

In [ ]:
print(vsc[0].name)
print(vsc[0].sample_img.shape)
print(vsc[0].cut_group)
print(vsc[0].scores_raw)

In [ ]:

sample_volume_names = [c[:-1] for c in contents]
vsc = [volumeStruct(name = name_,sample_img = img, cut_group=cg, scores_raw = np.array(ss),scores_norm=None) for name_,img, cg, ss in zip(sample_volume_names, sample_volume, vol_cut_group,vol_ss_keeper)]

In [ ]:
unique_group = np.unique(vol_cut_group)
grouped_vsc = {}
for gid in unique_group:
    grouped_vsc[gid] = [vs for vs in vsc if vs.cut_group == gid]

In [ ]:
grouped_vsc_sorted = {}
for gvsc_id in grouped_vsc:
    grouped_vsc_sorted[gvsc_id] = [vs for vs in sorted(grouped_vsc.get(gvsc_id), key=lambda vs: np.mean(vs.scores_raw), reverse=True)]

In [ ]:
for gid in unique_group:
    imshow_15("group " + str(gid), [vs.sample_img for vs in grouped_vsc_sorted[gid]], [vs.name for vs in grouped_vsc_sorted[gid]])

In [ ]:
default_sel_percent = 1.0
param_num = len(vol_ss_keeper[0])
grouped_vsc_sorted_norm = {}
for gvsc_id in grouped_vsc:
    gsvsc = grouped_vsc_sorted[gvsc_id]
    
    max_vs_local = []
    for i in range(param_num):
        max_vs_local.append(np.max([vs.scores_raw[i] for vs in gsvsc]))
    
    for i in range(len(gsvsc)):
        gsvsc[i] = gsvsc[i]._replace(scores_norm = [u/(v+0.0001) for u,v in zip(gsvsc[i].scores_raw,max_vs_local)])
        
    sel_num = int(len(gsvsc) * default_sel_percent)
    sel_gsvc = gsvsc[:sel_num]
    rule_out_gsvc = gsvsc[sel_num:]
    
    
    #sort_mean_sel_gsvc = [vs for vs in sorted(sel_gsvc, key=lambda vs: np.mean([u/(v+0.001) for u,v in zip(vs.scores_raw,max_vs_local)]), reverse=True)]
    
    sort_mean_sel_gsvc = [vs for vs in sorted(sel_gsvc, key=lambda vs: np.mean(vs.scores_norm), reverse=True)]
    grouped_vsc_sorted_norm[gvsc_id] = sort_mean_sel_gsvc + rule_out_gsvc

In [ ]:
for gid in unique_group:
    titles = [ str(round(np.mean(vs.scores_raw), 2))+'/'+str(round(np.mean(vs.scores_norm), 2)) + '\n' + vs.name for vs in grouped_vsc_sorted_norm[gid]]
    imshow_15('group ' +str(gid),[vs.sample_img for vs in grouped_vsc_sorted_norm[gid]], titles)

In [ ]:
save_thumbnails("report/Larry-2012-01-17-MRI", vd.raw_data)

In [ ]:
def imshow_single(img, title):
    fig,ax = plt.subplots()
    ax.imshow(img,cmap='gray')
    ax.set_title(title)
    plt.axis('off')

In [ ]:
# default_report_name = 'report.tsv'
# def export_report(report_path, scores):
#     report_path = os.path.join(report_path, default_report_namee)
#     #write headers
#     if not os.path.exists(report_path):
#         with open(report_path, 'w') as rf:
#             rf.write("#items: " + '\t'.join(measure_func.keys()) + '\n')
#     with open(report_path, 'a') as rf:
#         rf.write()
    
#     #write contents
    